## 순환신경망 구현 및 학습

In [1]:
import tensorflow as tf

## 하이퍼 파라미터

In [2]:
EPOCHS = 10
NUM_WORDS = 10000

## 모델 정의

In [6]:
# tensorflow 2.0방식으로 모델 정의하자
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__() # 상속받은 걸로 initailization
        
        # 입력으로 들어오게 될 것은 32길이의 숫자들
        # 사실 숫자로써의 의미는 없고 class라고 봐야 함
        # 따라서 숫자가 비슷하다고 더 비슷한 의미를 가지거나 그렇진 않아
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16) # 그래서 숫자들마다 독립적으로 사용되기위해 임베딩 해줌
        # 임베딩 : 단어나 문자(character)를 수치화
        # 정수 인덱스를 길이 n의 실수벡터로 맵핑. ex) n=2일때 [[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]
        # 차원이 16이 되는 임베딩, 즉 길이가 16인 벡터로 바꿔줘
        # 길이가 32인 숫자들이 16길이의 feature가 32개가 쭉 나열되어있는 것으로 바뀜
        
        self.rnn = tf.keras.layers.SimpleRNN(32) # 뉴런의 개수는 32개로 정함, Embedding에서 16으로 정해줬으니 2배정도
        self.dense = tf.keras.layers.Dense(1, activation='sigmoid') # dense layer를 만들어서 binary classification이니까 'sigmoid'로
        # 또는 tf.keras.layers.Dense(2, activation='softmax')로 해서
        # 모델 생성 부분에서 loss=categorical_crossentropy로 바꿔줘도 됨. 하지만 이론적으론 위가 맞음
        
    def __call__(self, x, training=None, mask=None): # 연결성
        x = self.emb(x)
        x = self.rnn(x)
        return self.dense(x)

### embedding 이해
![embedding](./embedding.PNG)

## IMDB 데이터셋 준비


In [4]:
# imdb dataset : 입력이 문장으로 되어있고, 정답은 0 또는 1(binary class)
# x : 영화에 대한 평가를 문장으로 나타낸 것
# y : 그 평이 부정/긍정인지 표현하는 label (0/1)

imdb = tf.keras.datasets.imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=NUM_WORDS)
# train같은 경우에 모든 단어들을 다 사용할 수 없기 때문에 제일 많이 들어있는 10000개(NUM_WORDS)의 단어만 사용하게끔

print(len(x_train[0]), len(x_train[1]))
# print(x_train) - 숫자 1로 시작 : sos(start of sequence)라는 특정한 단어로 모든 문장의 시작을 뜻함
# 그외 나머지는 영어 단어로 되어있는데, 가장 자주 쓰이는 단어들부터 2번부터 큰 순으로 배정되어있음

# x_train[0]과 x_train[1]의 길이 다름
# 이렇게 긴 문장들 중 전체를 다 쓰지않고 문장의 앞의 32단어만 보고 이를 classification하는 문제로 만들거임
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train,
                                                       value=0, # padding이 필요하면 0으로 
                                                       padding='pre',# padding이 필요하면 앞쪽으로 padding
                                                       maxlen=32) # 32길이로 잘라줌
                                                       # 32길이가 안되면 앞쪽에서 0으로 padding
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test,
                                                       value=0, 
                                                       padding='pre',
                                                       maxlen=32)

# 앞쪽은 전처리만 해준 것이고 dataset으로 만들어줘야 함
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

218 189


## 모델 생성

In [7]:
model = MyModel()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              # multi가 아닌 binary이기 때문에 sparse_categorical_crossentropy가 아닌 binary_crossentropy
              metrics=['accuracy'])

## 학습 루프 동작

In [8]:
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Train for 782 steps, validate for 782 steps
Epoch 1/10
782/782 [==============================] - 10s 13ms/step - loss: 0.5366 - accuracy: 0.7171 - val_loss: 0.4523 - val_accuracy: 0.7893
Epoch 2/10
782/782 [==============================] - 9s 12ms/step - loss: 0.3672 - accuracy: 0.8400 - val_loss: 0.4724 - val_accuracy: 0.7778
Epoch 3/10
782/782 [==============================] - 9s 12ms/step - loss: 0.2357 - accuracy: 0.9092 - val_loss: 0.5646 - val_accuracy: 0.7664
Epoch 4/10
782/782 [==============================] - 9s 12ms/step - loss: 0.1052 - accuracy: 0.9666 - val_loss: 0.7769 - val_accuracy: 0.7434
Epoch 5/10
782/782 [==============================] - 9s 12ms/step - loss: 0.0460 - accuracy: 0.9866 - val_loss: 0.9256 - val_accuracy: 0.7543
Epoch 6/10
782/782 [==============================] - 10s 12ms/step - loss: 0.0272 - accuracy: 0.9922 - val_loss: 1.0236 - val_accuracy: 0.7451
Epoch 7/10
782/782 [==============================] - 10s 12ms/step - loss: 0.0187 - accuracy: 0

**<LSTM 결과> - self.lstm = tf.keras.layers.LSTM(32)**
<br>
Train for 782 steps, validate for 782 steps
Epoch 1/10
782/782 [==============================] - 23s 29ms/step - loss: 0.5062 - accuracy: 0.7442 - val_loss: 0.4426 - val_accuracy: 0.7920
Epoch 2/10
782/782 [==============================] - 20s 25ms/step - loss: 0.3711 - accuracy: 0.8348 - val_loss: 0.4599 - val_accuracy: 0.7890
Epoch 3/10
782/782 [==============================] - 20s 26ms/step - loss: 0.3148 - accuracy: 0.8657 - val_loss: 0.4928 - val_accuracy: 0.7808
Epoch 4/10
782/782 [==============================] - 22s 28ms/step - loss: 0.2608 - accuracy: 0.8944 - val_loss: 0.5275 - val_accuracy: 0.7706
Epoch 5/10
782/782 [==============================] - 22s 28ms/step - loss: 0.2116 - accuracy: 0.9168 - val_loss: 0.5966 - val_accuracy: 0.7702
Epoch 6/10
782/782 [==============================] - 19s 24ms/step - loss: 0.1680 - accuracy: 0.9369 - val_loss: 0.7475 - val_accuracy: 0.7612
Epoch 7/10
782/782 [==============================] - 17s 22ms/step - loss: 0.1336 - accuracy: 0.9526 - val_loss: 0.7777 - val_accuracy: 0.7537
Epoch 8/10
782/782 [==============================] - 14s 18ms/step - loss: 0.1079 - accuracy: 0.9611 - val_loss: 1.0962 - val_accuracy: 0.7492
Epoch 9/10
782/782 [==============================] - 21s 27ms/step - loss: 0.0851 - accuracy: 0.9697 - val_loss: 1.0680 - val_accuracy: 0.7543
Epoch 10/10
782/782 [==============================] - 17s 22ms/step - loss: 0.0678 - accuracy: 0.9772 - val_loss: 1.1366 - val_accuracy: 0.7489

- training loss는 점점 줄어드는데, validation loss는 오히려 증가하는 중(안 좋아지고있지)
- 이는 overfitting이 발생했다고 봄
- overfitting 방지로 LSTM대신 GRU를 한번 사용해보자

**<GRU 결과> - self.gru = tf.keras.layers.GRU(32)**
<br>
Train for 782 steps, validate for 782 steps
Epoch 1/10
782/782 [==============================] - 14s 17ms/step - loss: 0.0517 - accuracy: 0.9819 - val_loss: 1.3653 - val_accuracy: 0.7424
Epoch 2/10
782/782 [==============================] - 15s 19ms/step - loss: 0.0437 - accuracy: 0.9846 - val_loss: 1.3452 - val_accuracy: 0.7484
Epoch 3/10
782/782 [==============================] - 13s 17ms/step - loss: 0.0367 - accuracy: 0.9886 - val_loss: 1.5042 - val_accuracy: 0.7469
Epoch 4/10
782/782 [==============================] - 14s 18ms/step - loss: 0.0342 - accuracy: 0.9888 - val_loss: 1.4369 - val_accuracy: 0.7464
Epoch 5/10
782/782 [==============================] - 22s 28ms/step - loss: 0.0249 - accuracy: 0.9918 - val_loss: 1.5078 - val_accuracy: 0.7434
Epoch 6/10
782/782 [==============================] - 24s 31ms/step - loss: 0.0211 - accuracy: 0.9934 - val_loss: 1.6381 - val_accuracy: 0.7478
Epoch 7/10
782/782 [==============================] - 20s 26ms/step - loss: 0.0251 - accuracy: 0.9924 - val_loss: 1.7432 - val_accuracy: 0.7440
Epoch 8/10
782/782 [==============================] - 15s 19ms/step - loss: 0.0167 - accuracy: 0.9946 - val_loss: 1.7237 - val_accuracy: 0.7466
Epoch 9/10
782/782 [==============================] - 14s 18ms/step - loss: 0.0184 - accuracy: 0.9940 - val_loss: 1.8408 - val_accuracy: 0.7456
Epoch 10/10
782/782 [==============================] - 14s 18ms/step - loss: 0.0161 - accuracy: 0.9951 - val_loss: 1.8500 - val_accuracy: 0.7492

- GRU를 사용해도 여전히 validation loss는 줄어들지않고 overfitting이 일어남
- Vanilla RNN도 해보자

**<RNN 결과> - self.rnn = tf.keras.layers.SimpleRNN(32)**
<br>
Train for 782 steps, validate for 782 steps
Epoch 1/10
782/782 [==============================] - 10s 13ms/step - loss: 0.5366 - accuracy: 0.7171 - val_loss: 0.4523 - val_accuracy: 0.7893
Epoch 2/10
782/782 [==============================] - 9s 12ms/step - loss: 0.3672 - accuracy: 0.8400 - val_loss: 0.4724 - val_accuracy: 0.7778
Epoch 3/10
782/782 [==============================] - 9s 12ms/step - loss: 0.2357 - accuracy: 0.9092 - val_loss: 0.5646 - val_accuracy: 0.7664
Epoch 4/10
782/782 [==============================] - 9s 12ms/step - loss: 0.1052 - accuracy: 0.9666 - val_loss: 0.7769 - val_accuracy: 0.7434
Epoch 5/10
782/782 [==============================] - 9s 12ms/step - loss: 0.0460 - accuracy: 0.9866 - val_loss: 0.9256 - val_accuracy: 0.7543
Epoch 6/10
782/782 [==============================] - 10s 12ms/step - loss: 0.0272 - accuracy: 0.9922 - val_loss: 1.0236 - val_accuracy: 0.7451
Epoch 7/10
782/782 [==============================] - 10s 12ms/step - loss: 0.0187 - accuracy: 0.9948 - val_loss: 1.1551 - val_accuracy: 0.7316
Epoch 8/10
782/782 [==============================] - 9s 12ms/step - loss: 0.0163 - accuracy: 0.9946 - val_loss: 1.2417 - val_accuracy: 0.7390
Epoch 9/10
782/782 [==============================] - 15s 19ms/step - loss: 0.0187 - accuracy: 0.9938 - val_loss: 1.2685 - val_accuracy: 0.7325
Epoch 10/10
782/782 [==============================] - 16s 21ms/step - loss: 0.0121 - accuracy: 0.9964 - val_loss: 1.3621 - val_accuracy: 0.7327

- 여전히 overfitting
- 이 데이터셋은 rnn을 사용하는게 효과적으로 나타나진 않음
- 이런 경우에는 attention layer를 추가한다는 등 성능을 더 좋게 할 수 있음
- 지금은 배우지 않았기때문에 여기서 끝냄
- 나중에 attention network를 사용해서 성능이 얼마나 나오는지 확인해보자